# Import modules and define functions

In [4]:
import glob
import holoviews as hv
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from pandas import Series, DataFrame, Panel 
import xarray as xr 

Calculates daily climatology with a 15-day rolling window 

In [5]:
def daily_climatology (tmax):
    
    rolling_mean_dailytmax = tmax.rolling(time=15, center=True).mean()
    
    climatology = rolling_mean_dailytmax.groupby('time.dayofyear').mean('time')
    
    return climatology 

Calculates daily climatology of 90th percentile with a 15-day rolling window 

In [6]:
def climatology_90 (tmax):
    rolling_mean_dailytmax = tmax.rolling(time=15, center=True).mean()
    percentile_90 = rolling_mean_dailytmax.groupby('time.dayofyear').quantile(0.9)
    
    return percentile_90

Identifies consecutive days on which the daily tmax exceeded the threshold 

In [7]:
def consecutive(data, stepsize=1):
    return np.split(data, np.where(np.diff(data) != stepsize)[0]+1)

Filters out events that lasted for a minimum of 3 days and returns day of the year and the duration of an event #doesn't work as expected 

In [8]:
def find_heat(x):
    if x.size >= 3:
        return x[0], x.size

In [9]:
def duration (events):
    return events[0], events.size   #problem with this function is that there is no iteration 

In [10]:
def duration2 (events):
    return [(x[0], x.size) for x in events]

# Opening yearly ERA5 surface Tmax files and dropped leap days

In [11]:
files = sorted(glob.glob('/g/data/e14/cp3790/Charuni/ERA5-new/era5_dailytmax_*.nc'))

#Slice the data for single lat lon in NSW
era5_dailytmax_aus = xr.open_mfdataset(files, combine='by_coords').sel(time=slice('1982-12-25', '2013-01-07'), longitude=141.25, latitude=-33.75).load()
daily_tmax = era5_dailytmax_aus["dmax"].sel(time=~((era5_dailytmax_aus["dmax"].time.dt.month == 2) & (era5_dailytmax_aus["dmax"].time.dt.day == 29)))
daily_tmax.attrs['units'] = 'deg C'

# Code

## For single year

Calculate daily mean climatology and 90th percentile 

In [12]:
daily_mean = daily_climatology (daily_tmax)
threshold = climatology_90 (daily_tmax)

Select one year (2010 in this case) and compare each day of the year against the corresponding threshold value and return boolean 1s and 0s.

The days on which the daily tmax exceeded the threshold are then filtered out.

Consecutive days are identified 

In [14]:
year_2010 = daily_tmax.sel(time='2010')
heatwave_days = (year_2010.groupby('time.dayofyear') > threshold).astype(int)

In [15]:
heatwave_days

<xarray.DataArray (time: 365)>
array([1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1])
Coordinates:
    quantile   float64 0.9
    longitude  float32 141.25
    latitude   float32 -33.75
  * time       (time) datetime64[ns] 2010-01-01 2010-01-02 ... 2010-12-31
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 358 359 360 361 362 363 364 365

In [17]:
heatwave_events = np.where(heatwave_days==1)
heatwave_events

(array([  0,   6,   7,   8,   9,  10,  11,  19,  20,  21,  24,  25,  29,
         30,  31,  32,  37,  38,  39,  40,  41,  47,  48,  49,  50,  51,
         55,  56,  57,  73,  74,  75,  76,  77,  78,  79,  83,  84,  85,
         86,  94, 106, 107, 108, 109, 110, 111, 112, 120, 122, 123, 128,
        129, 166, 167, 168, 169, 170, 173, 174, 190, 193, 209, 210, 229,
        242, 275, 276, 277, 278, 282, 283, 284, 294, 301, 310, 312, 313,
        314, 315, 324, 337, 338, 339, 340, 362, 363, 364]),)

In [19]:
counters = consecutive(heatwave_events[0].tolist())
counters

[array([0]),
 array([ 6,  7,  8,  9, 10, 11]),
 array([19, 20, 21]),
 array([24, 25]),
 array([29, 30, 31, 32]),
 array([37, 38, 39, 40, 41]),
 array([47, 48, 49, 50, 51]),
 array([55, 56, 57]),
 array([73, 74, 75, 76, 77, 78, 79]),
 array([83, 84, 85, 86]),
 array([94]),
 array([106, 107, 108, 109, 110, 111, 112]),
 array([120]),
 array([122, 123]),
 array([128, 129]),
 array([166, 167, 168, 169, 170]),
 array([173, 174]),
 array([190]),
 array([193]),
 array([209, 210]),
 array([229]),
 array([242]),
 array([275, 276, 277, 278]),
 array([282, 283, 284]),
 array([294]),
 array([301]),
 array([310]),
 array([312, 313, 314, 315]),
 array([324]),
 array([337, 338, 339, 340]),
 array([362, 363, 364])]

Events that last for at least 3 consecutive days are filtered and assigned to new array 'hottest' and gives us the start day of event and the number of days 

In [20]:
hottest_single = [(x[0], x.size) for x in counters if x.size >= 3]

In [21]:
hottest_single

[(6, 6),
 (19, 3),
 (29, 4),
 (37, 5),
 (47, 5),
 (55, 3),
 (73, 7),
 (83, 4),
 (106, 7),
 (166, 5),
 (275, 4),
 (282, 3),
 (312, 4),
 (337, 4),
 (362, 3)]

## For multiple years 

In [18]:
multiple_years = daily_tmax.sel(time=slice('2010', '2012'))